## Requirment Libraries

In [1]:
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchWindowException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import numpy as np


## Selenium Web Driver Setup

In [2]:
service = Service('C:/Users/fb_19/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')  # starts the browser in maximized window

options.add_argument('--disable-extensions')  # disables browser extensions
options.add_argument('--disable-gpu')  # disables hardware acceleration
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)


## Definition Of Element Lists

In [3]:

product_links = []
product_names = []
reviews = []
ids=[]
product_ratings=[]
product_categories=[]

## Reforming a Dataframe

In [4]:
df = pd.DataFrame(columns=("product-id","product-name","product-links","product-category","product-reviews","product-rating"))


## Collecting Product ID numbers

Define the categories that you need to scrap reviews. You can add links sorted by reviews, so this model can get more reviews per second.
You can change number of pages



In [ ]:
websites=("https://www.n11.com/evcil-hayvan-urunleri?srt=REVIEWS&pg=","https://www.n11.com/ayakkabi-ve-canta?srt=REVIEWS&pg=","https://www.n11.com/agiz-dis-bakimi?srt=REVIEWS&pg=","https://www.n11.com/elektrikli-ev-aletleri?srt=REVIEWS&pg=","https://www.n11.com/bebek-bezi-ve-islak-mendil?srt=REVIEWS&pg=")
for a in websites:
    for i in range(1, 5):
        try:
            website= a
            website += str(i)
            driver.get(website)
            website = a
            names = driver.find_elements(By.CLASS_NAME, 'productName')
            time.sleep(1)
            for name in names:
                product_names.append(name.text)
            # Wait for the columns to load
                wait = WebDriverWait(driver, 10)
                columns = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'column')))

            for column in columns:
                
                column_contents = column.find_elements(By.CLASS_NAME, 'columnContent')
                
                for content in column_contents:
                    
                    id_element = content.find_element(By.TAG_NAME, "a")
                    id = id_element.get_attribute("data-id")
                    product_link = id_element.get_attribute("href")
                    try:
                        category = driver.find_element(By.XPATH, '//*[@id="breadCrumb"]/ul/li[3]/a')
                    except NoSuchElementException:
                        category =driver.find_element(By.XPATH, '//*[@id="breadCrumb"]/ul/li[2]/a')

                    if id is not None:

                        ids.append(id)
                        product_links.append(product_link)
                        product_categories.append(category.text)
                    else:
                        continue


        except TimeoutException:
            print("TimeoutException error ignored.")
            continue
    
print(len(ids))

# Collecting Reviews

In [ ]:
for a in ids:
    i=1
    website = "https://www.n11.com/component/render/groupProductReviews?page="+str(i)+"&productId="+str(a)
    driver.get(website)
    time.sleep(0.1)
    try:
        try:
            page= int(driver.find_element(By.XPATH, '//*[@id="reviewContent"]/div/div/span').text)
        except NoSuchElementException:
            page=1
        for i in range (1,page+1):
            website = "https://www.n11.com/component/render/groupProductReviews?page="+str(i)+"&productId="+str(a)
            driver.get(website)
            step1 = driver.find_elements(By.CLASS_NAME,'comment')
            for step in step1:
                try:
                    comment = step.find_element(By.TAG_NAME,'p').text
                    reviews.append(comment)
                    rating = step.find_element(By.CLASS_NAME,'commentTop').find_element(By.CLASS_NAME,'ratingCont').find_element(By.TAG_NAME,'span').get_attribute("class")
                    rating = int(rating.split("rating r")[1])/20
                    product_ratings.append(rating)
                    df.loc[len(df)]=[a,product_names[ids.index(a)],product_links[ids.index(a)],product_categories[ids.index(a)],reviews[-1],product_ratings[-1]]
                except IndexError:
                    continue
    except TimeoutException:
        continue
        

In [ ]:
len(reviews),len(product_ratings)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df["product-reviews"].duplicated().value_counts()

In [ ]:
df["product-rating"].value_counts()

In [ ]:
df["product-reviews"]

In [ ]:
data = df.copy()
data=data.drop_duplicates("product-reviews")

In [1]:
data.to_csv('data-dropped-duplicated-reviews3.csv')

NameError: name 'data' is not defined

In [ ]:
data.info()

In [ ]:
data["product-category"].value_counts()